# Video Generation Pipeline v3 — Hierarchical (Topic → Chapters → Scenes)

**Architecture:**
- **LLM Planner** (llama3:8b): Generates a chapter-level blueprint from the topic
- **LLM Expander** (llama3:8b): Expands each chapter into detailed scenes with 2–3 image prompts
- **Stable Diffusion v1-5 + LCM LoRA**: Generates 2–3 visuals per scene (fast, 10 steps)
- **Edge-TTS**: Narration audio per scene
- **MoviePy**: Renders individual scene clips (image + audio)
- **FFmpeg**: Stream-copy merges clips → chapters → final video (no re-encode, fast)

## Cell 1: Imports & Configuration

In [1]:
import json
import os
import subprocess
import textwrap
import requests
import asyncio
from PIL import Image, ImageDraw, ImageFont
import edge_tts
from moviepy import ImageClip, AudioFileClip
import torch
from diffusers import StableDiffusionPipeline, LCMScheduler

# ── LLM Config ─────────────────────────────────────────────
OLLAMA_API_URL = "http://localhost:11434/api/generate"
OLLAMA_MODEL   = "llama3:8b"          # GPU-backed model

# ── Output Directories ──────────────────────────────────────
OUTPUT_DIR   = os.path.abspath("output_v3")  # absolute path avoids FFmpeg concat doubling
CHAPTERS_DIR = os.path.join(OUTPUT_DIR, "chapters")
FINAL_VIDEO  = os.path.join(OUTPUT_DIR, "final_video.mp4")

os.makedirs(CHAPTERS_DIR, exist_ok=True)

# ── Slide Dimensions ───────────────────────────────────────
SLIDE_W, SLIDE_H = 1280, 720

print("✅ Config ready.")
print(f"PyTorch: {torch.__version__} | CUDA: {torch.cuda.is_available()}")

c:\Users\navee\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Config ready.
PyTorch: 2.5.1+cu121 | CUDA: True


## Cell 2: Stable Diffusion Init (SD v1-5 + LCM LoRA)

In [2]:
MODEL_ID = "runwayml/stable-diffusion-v1-5"
LORA_ID  = "latent-consistency/lcm-lora-sdv1-5"

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading Stable Diffusion on {device}...")

pipe = StableDiffusionPipeline.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16 if device == "cuda" else torch.float32,
    safety_checker=None
).to(device)

# Memory optimisations
pipe.enable_attention_slicing()
pipe.enable_vae_slicing()
if device == "cuda":
    pipe.enable_model_cpu_offload()

# LCM LoRA — fast inference (10 steps)
pipe.load_lora_weights(LORA_ID)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)

def generate_image_sd(prompt_text: str, output_path: str) -> str | None:
    """Generate a flat educational illustration using SD + LCM LoRA."""
    full_prompt = (
        f"simple flat illustration of {prompt_text}, "
        "minimal design, clean white background, "
        "educational graphic, vector style, no text"
    )
    try:
        image = pipe(
            prompt=full_prompt,
            num_inference_steps=10,
            guidance_scale=1.5,
            height=512,
            width=512
        ).images[0]
        image.save(output_path)
        return output_path
    except Exception as e:
        print(f"  ⚠️  SD error: {e}")
        return None

print("✅ Stable Diffusion ready.")

Loading Stable Diffusion on cuda...


Loading pipeline components...: 100%|██████████| 6/6 [00:07<00:00,  1.21s/it]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


✅ Stable Diffusion ready.


## Step 1: LLM Planner — Chapter Blueprint

In [3]:
def generate_blueprint(topic: str) -> dict | None:
    """
    Single LLM call: converts a topic into a high-level chapter plan.
    Returns: { "title": str, "chapters": [ { "chapter_title", "chapter_summary", "key_themes" } ] }
    """
    prompt = f"""You are a video script planner.
Convert this topic into a structured chapter plan for an educational video.
Topic: {topic}

Return ONLY valid JSON (no markdown, no extra text):
{{
  "title": "<overall video title>",
  "chapters": [
    {{
      "chapter_title": "<chapter title>",
      "chapter_summary": "<one sentence summary>",
      "key_themes": ["theme1", "theme2", "theme3"]
    }}
  ]
}}

Generate 3 to 5 chapters. Return ONLY the JSON object."""

    print(f"📋 Planning chapters for: {topic}...")
    try:
        resp = requests.post(OLLAMA_API_URL, json={
            "model": OLLAMA_MODEL,
            "prompt": prompt,
            "format": "json",
            "stream": False
        }, timeout=120)
        resp.raise_for_status()
        data = json.loads(resp.json()["response"])
        print(f"  ✅ Blueprint ready: {len(data.get('chapters', []))} chapters")
        return data
    except requests.RequestException as e:
        print(f"  ❌ Ollama connection error: {e}")
        return None
    except (json.JSONDecodeError, KeyError) as e:
        print(f"  ❌ JSON parse error: {e}")
        return None

## Step 2: LLM Expander — Scene Details per Chapter

In [4]:
def expand_chapter(chapter: dict) -> dict | None:
    """
    Second LLM call: expands a chapter summary into 3–5 detailed scenes.
    Each scene has 2 image_prompts for Stable Diffusion.
    Returns: { "scenes": [ { "title", "bullets", "narration", "image_prompts": [] } ] }
    """
    prompt = f"""You are a video scriptwriter.
Expand this chapter into detailed scenes for an educational video.

Chapter Title: {chapter['chapter_title']}
Chapter Summary: {chapter['chapter_summary']}
Key Themes: {', '.join(chapter.get('key_themes', []))}

Return ONLY valid JSON (no markdown, no extra text):
{{
  "scenes": [
    {{
      "title": "<scene title>",
      "bullets": ["<key point 1>", "<key point 2>", "<key point 3>"],
      "narration": "<60-90 word spoken narration for this scene>",
      "image_prompts": [
        "<visual description 1 for illustration>",
        "<visual description 2 for illustration>"
      ]
    }}
  ]
}}

Generate 3 to 5 scenes. Each scene must have exactly 2 image_prompts. Return ONLY the JSON object."""

    print(f"  📝 Expanding: {chapter['chapter_title']}...")
    try:
        resp = requests.post(OLLAMA_API_URL, json={
            "model": OLLAMA_MODEL,
            "prompt": prompt,
            "format": "json",
            "stream": False
        }, timeout=180)
        resp.raise_for_status()
        data = json.loads(resp.json()["response"])
        print(f"    ✅ {len(data.get('scenes', []))} scenes generated")
        return data
    except requests.RequestException as e:
        print(f"    ❌ Ollama error: {e}")
        return None
    except (json.JSONDecodeError, KeyError) as e:
        print(f"    ❌ JSON error: {e}")
        return None

## Step 3: Generate Scene Visuals (2 SD images per scene)

In [5]:
def generate_scene_images(scene: dict, scenes_dir: str, ch_i: int, sc_i: int) -> list[str]:
    """
    Generate 2 SD images for a scene. Returns list of valid image paths.
    """
    image_prompts = scene.get("image_prompts", [])
    paths = []
    for img_i, prompt in enumerate(image_prompts[:2]):  # cap at 2
        out_path = os.path.join(scenes_dir, f"ch{ch_i}_sc{sc_i}_img{img_i}.png")
        print(f"    🎨 Generating image {img_i+1}/2: {prompt[:50]}...")
        result = generate_image_sd(prompt, out_path)
        if result:
            paths.append(result)
    return paths

## Step 4: Slide Renderer (PIL) — Title + Bullets + 2 Tiled Images

In [6]:
def create_slide(scene: dict, image_paths: list[str], scenes_dir: str, ch_i: int, sc_i: int) -> str:
    """
    Renders a 1280x720 slide:
      - Left half: title + bullet points
      - Right half: up to 2 SD images stacked vertically
    """
    img = Image.new("RGB", (SLIDE_W, SLIDE_H), color="#F8F9FA")
    draw = ImageDraw.Draw(img)

    # ── Fonts ───────────────────────────────────────────────
    try:
        title_font = ImageFont.truetype("arial.ttf", 52)
        body_font  = ImageFont.truetype("arial.ttf", 32)
    except:
        title_font = ImageFont.load_default()
        body_font  = ImageFont.load_default()

    margin       = 50
    text_width   = 580  # left half
    right_start  = 660  # right half x start
    right_width  = SLIDE_W - right_start - margin

    # ── Draw blue left accent bar ────────────────────────────
    draw.rectangle([(margin - 10, 40), (margin - 4, SLIDE_H - 40)], fill="#4A90D9")

    # ── Draw Title ───────────────────────────────────────────
    title = scene.get("title", f"Scene {sc_i}")
    wrapped_title = textwrap.wrap(title, width=22)
    ty = 60
    for line in wrapped_title:
        draw.text((margin, ty), line, fill="#1A1A2E", font=title_font)
        ty += 62

    # ── Draw Bullets ─────────────────────────────────────────
    y = ty + 20
    for bullet in scene.get("bullets", []):
        for line in textwrap.wrap(bullet, width=32):
            draw.text((margin + 20, y), f"• {line}", fill="#333333", font=body_font)
            y += 42
        y += 8  # extra gap between bullets

    # ── Paste SD Images (right half, stacked) ────────────────
    if image_paths:
        num_imgs = len(image_paths)
        slot_h   = (SLIDE_H - 2 * margin) // num_imgs

        for idx, img_path in enumerate(image_paths):
            try:
                sd_img = Image.open(img_path)
                # Fit into slot while preserving aspect ratio
                aspect   = sd_img.width / sd_img.height
                fit_h    = min(slot_h - 20, 320)
                fit_w    = int(fit_h * aspect)
                if fit_w > right_width:
                    fit_w = right_width
                    fit_h = int(fit_w / aspect)
                sd_img = sd_img.resize((fit_w, fit_h), Image.Resampling.LANCZOS)
                # Centre horizontally in right zone
                x_pos = right_start + (right_width - fit_w) // 2
                y_pos = margin + idx * slot_h + (slot_h - fit_h) // 2
                img.paste(sd_img, (x_pos, y_pos))
            except Exception as e:
                print(f"    ⚠️  Could not paste image {idx}: {e}")

    # ── Save ─────────────────────────────────────────────────
    slide_path = os.path.join(scenes_dir, f"ch{ch_i}_sc{sc_i}_slide.png")
    img.save(slide_path)
    return slide_path

## Step 5: Audio Generation (Edge-TTS)

In [7]:
async def generate_audio(text: str, audio_dir: str, ch_i: int, sc_i: int) -> str | None:
    """
    Converts narration text to speech using Edge-TTS.
    Voice: en-US-ChristopherNeural (male, GPU-friendly quality)
    """
    voice       = "en-US-ChristopherNeural"
    output_file = os.path.join(audio_dir, f"ch{ch_i}_sc{sc_i}.mp3")
    print(f"    🔊 Generating audio for ch{ch_i} sc{sc_i}...")
    try:
        communicate = edge_tts.Communicate(text, voice)
        await communicate.save(output_file)
        return output_file
    except Exception as e:
        print(f"    ❌ TTS error: {e}")
        return None

## Step 6: Scene Clip Renderer (MoviePy — single clip only)

In [8]:
def render_scene_clip(slide_path: str, audio_path: str, clips_dir: str, ch_i: int, sc_i: int) -> str | None:
    """
    Combines slide image + audio into a scene MP4 clip.
    Duration is driven by the audio length. 24 fps, H.264/AAC.
    """
    output_path = os.path.join(clips_dir, f"ch{ch_i}_sc{sc_i}.mp4")
    print(f"    🎬 Rendering clip: ch{ch_i}_sc{sc_i}.mp4")
    try:
        audio = AudioFileClip(audio_path)
        clip  = ImageClip(slide_path).with_duration(audio.duration).with_audio(audio)
        clip.write_videofile(
            output_path,
            fps=24,
            codec="libx264",
            audio_codec="aac",
            logger=None       # suppress MoviePy progress bars
        )
        audio.close()
        clip.close()
        return output_path
    except Exception as e:
        print(f"    ❌ MoviePy error: {e}")
        return None

## Step 7 & 8: FFmpeg Merging (Chapter + Final)

In [9]:
def ffmpeg_concat(clip_paths: list[str], output_path: str) -> str | None:
    """
    Generic FFmpeg stream-copy concatenator.
    Writes a concat list file then calls ffmpeg -f concat -safe 0 -i list -c copy output.
    Stream-copy = no re-encode → extremely fast and supports arbitrarily long videos.
    """
    if not clip_paths:
        print("  ⚠️  No clips to merge.")
        return None

    list_file = output_path.replace(".mp4", "_list.txt")
    with open(list_file, "w", encoding="utf-8") as f:
        for path in clip_paths:
            # FFmpeg concat list requires forward slashes
            abs_path = os.path.abspath(path).replace(chr(92), '/')
            f.write(f"file '{abs_path}'\n")

    cmd = [
        "ffmpeg", "-y",
        "-f", "concat",
        "-safe", "0",
        "-i", list_file,
        "-c", "copy",
        output_path
    ]
    print(f"  🔗 FFmpeg merging {len(clip_paths)} clips → {os.path.basename(output_path)}")
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode != 0:
        print(f"  ❌ FFmpeg error:\n{result.stderr[-500:]}")
        return None
    print(f"  ✅ Merged: {output_path}")
    return output_path


def merge_chapter(ch_i: int, clip_paths: list[str], chapter_dir: str) -> str | None:
    """Step 7: Merge all scene clips of a chapter into chapter_i.mp4"""
    out = os.path.join(chapter_dir, f"chapter_{ch_i}.mp4")
    return ffmpeg_concat(clip_paths, out)


def merge_final(chapter_paths: list[str]) -> str | None:
    """Step 8: Merge all chapter videos into final_video.mp4"""
    return ffmpeg_concat(chapter_paths, FINAL_VIDEO)

## Execution Pipeline — `main(topic)`

In [10]:
async def main(topic: str):
    print(f"\n{'='*60}")
    print(f"🚀 Starting Video Generation: '{topic}'")
    print(f"{'='*60}\n")

    # ── Step 1: Blueprint ───────────────────────────────────
    blueprint = generate_blueprint(topic)
    if not blueprint:
        print("❌ Blueprint generation failed. Aborting.")
        return

    # Save blueprint for reference
    blueprint_path = os.path.join(OUTPUT_DIR, "video_blueprint.json")
    with open(blueprint_path, "w") as f:
        json.dump(blueprint, f, indent=2)
    print(f"📄 Blueprint saved → {blueprint_path}\n")

    chapters       = blueprint.get("chapters", [])
    chapter_videos = []   # final chapter MP4 paths

    for ch_i, chapter in enumerate(chapters, start=1):
        ch_title = chapter.get("chapter_title", f"Chapter {ch_i}")
        print(f"\n{'─'*50}")
        print(f"📖 Chapter {ch_i}: {ch_title}")
        print(f"{'─'*50}")

        # ── Per-chapter directories ────────────────────────
        ch_dir     = os.path.join(CHAPTERS_DIR, f"ch_{ch_i}")
        scenes_dir = os.path.join(ch_dir, "scenes")
        audio_dir  = os.path.join(ch_dir, "audio")
        clips_dir  = os.path.join(ch_dir, "clips")
        for d in [scenes_dir, audio_dir, clips_dir]:
            os.makedirs(d, exist_ok=True)

        # ── Step 2: Expand chapter → scenes ───────────────
        expansion = expand_chapter(chapter)
        if not expansion:
            print(f"  ⚠️  Skipping chapter {ch_i} — expansion failed.")
            continue

        scenes      = expansion.get("scenes", [])
        scene_clips = []   # scene MP4 paths for this chapter

        for sc_i, scene in enumerate(scenes, start=1):
            sc_title = scene.get("title", f"Scene {sc_i}")
            print(f"\n  🎥 Scene {sc_i}/{len(scenes)}: {sc_title}")

            # ── Step 3: Generate 2 SD visuals ─────────────
            image_paths = generate_scene_images(scene, scenes_dir, ch_i, sc_i)

            # ── Step 4: Render slide ───────────────────────
            slide_path = create_slide(scene, image_paths, scenes_dir, ch_i, sc_i)

            # ── Step 5: Generate narration audio ──────────
            narration = scene.get("narration", "").strip()
            if not narration:
                print(f"    ⚠️  No narration for sc{sc_i} — skipping.")
                continue

            audio_path = await generate_audio(narration, audio_dir, ch_i, sc_i)
            if not audio_path:
                continue

            # ── Step 6: Render scene clip ──────────────────
            clip_path = render_scene_clip(slide_path, audio_path, clips_dir, ch_i, sc_i)
            if clip_path:
                scene_clips.append(clip_path)

        # ── Step 7: FFmpeg merge → chapter video ──────────
        if scene_clips:
            print(f"\n  ► Merging {len(scene_clips)} scene clips for chapter {ch_i}...")
            ch_video = merge_chapter(ch_i, scene_clips, ch_dir)
            if ch_video:
                chapter_videos.append(ch_video)
        else:
            print(f"  ⚠️  No clips for chapter {ch_i}.")

    # ── Step 8: FFmpeg merge → final video ──────────────────
    print(f"\n{'='*60}")
    if chapter_videos:
        print(f"🎞️  Merging {len(chapter_videos)} chapters into final video...")
        final = merge_final(chapter_videos)
        if final:
            print(f"\n🎉 Done! Final video → {final}")
    else:
        print("❌ No chapter videos created — nothing to merge.")
    print(f"{'='*60}\n")

## ▶️ Run — Change the topic below!

In [12]:
await main("The history of Chess")


🚀 Starting Video Generation: 'The history of Chess'

📋 Planning chapters for: The history of Chess...
  ✅ Blueprint ready: 4 chapters
📄 Blueprint saved → d:\AI-powered-Learning-platform\notebooks\output_v3\video_blueprint.json


──────────────────────────────────────────────────
📖 Chapter 1: Ancient Origins and Early Development
──────────────────────────────────────────────────
  📝 Expanding: Ancient Origins and Early Development...
    ✅ 4 scenes generated

  🎥 Scene 1/4: The Birthplace of Chess
    🎨 Generating image 1/2: An illustration of ancient Indian cities like Vara...


100%|██████████| 10/10 [00:09<00:00,  1.05it/s]


    🎨 Generating image 2/2: A depiction of Indian soldiers playing chess durin...


100%|██████████| 10/10 [00:06<00:00,  1.66it/s]


    🔊 Generating audio for ch1 sc1...
    🎬 Rendering clip: ch1_sc1.mp4

  🎥 Scene 2/4: Chess in Ancient Persia
    🎨 Generating image 1/2: A scene of camel caravans crossing the desert...


...

    🔊 Generating audio for ch4 sc3...
    🎬 Rendering clip: ch4_sc3.mp4

  🎥 Scene 4/4: Global Connectivity
    🎨 Generating image 1/2: A illustration of a global chess network with conn...


100%|██████████| 10/10 [00:05<00:00,  1.68it/s]


    🎨 Generating image 2/2: A scene showing a online chess tournament in progr...


100%|██████████| 10/10 [00:05<00:00,  1.68it/s]


    🔊 Generating audio for ch4 sc4...
    🎬 Rendering clip: ch4_sc4.mp4

  ► Merging 4 scene clips for chapter 4...
  🔗 FFmpeg merging 4 clips → chapter_4.mp4
  ✅ Merged: d:\AI-powered-Learning-platform\notebooks\output_v3\chapters\ch_4\chapter_4.mp4

🎞️  Merging 4 chapters into final video...
  🔗 FFmpeg merging 4 clips → final_video.mp4
  ✅ Merged: d:\AI-powered-Learning-platform\notebooks\output_v3\final_video.mp4

🎉 Done! Final video → d:\AI-powered-Learning-platform\notebooks\output_v3\final_video.mp4

